In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from pyomo.environ import *
from pyomo.network import *
from idaes.core.util.model_statistics import degrees_of_freedom
from Method import Initialize
from Unit_Model import *

In [2]:
# Create a concrete model
m = ConcreteModel()

# Define a set for components
m.Phase = Set(initialize = ['Liq','Vap'])
m.Comp = Set(initialize = ['benzene','toluene','methane','hydrogen'])
m.R101 = R101_creater(m)
m.H101 = H101_creater(m)
m.F101 = F101_creater(m)
m.F102 = F102_creater(m)
m.M101 = M101_creater(m)
m.S101 = S101_creater(m)

In [3]:
m.s1 = Arc(source=m.H101.Outlet, destination=m.R101.Inlet)
m.s2 = Arc(source=m.R101.Outlet, destination=m.F101.Inlet)
m.s3 = Arc(source=m.F101.Liq_Outlet, destination=m.F102.Inlet)
m.s4 = Arc(source=m.F101.Vap_Outlet, destination=m.S101.Inlet)
m.s5 = Arc(source=m.S101.Recycle, destination=m.M101.Recycle_Feed)
m.s6 = Arc(source=m.M101.Outlet, destination=m.H101.Inlet)
TransformationFactory("network.expand_arcs").apply_to(m)

In [4]:
m.H101.outlet.P.fix(350000)
m.H101.outlet.T.fix(600)

m.F101.outlet.P.fix(350000)
m.F101.outlet.T.fix(325)

m.F102.outlet.P.fix(150000)
m.F102.outlet.T.fix(375)

m.R101.Q.fix(0)

In [5]:
# 各组分进料
m.M101.inlet_feed['Toluene'].flow['Liq','benzene'].fix(1e-8)
m.M101.inlet_feed['Toluene'].flow['Liq','toluene'].fix(0.30)
m.M101.inlet_feed['Toluene'].flow['Liq','methane'].fix(1e-8)
m.M101.inlet_feed['Toluene'].flow['Liq','hydrogen'].fix(1e-8)
m.M101.inlet_feed['Toluene'].flow['Vap','benzene'].fix(1e-8)
m.M101.inlet_feed['Toluene'].flow['Vap','toluene'].fix(1e-8)
m.M101.inlet_feed['Toluene'].flow['Vap','methane'].fix(1e-8)
m.M101.inlet_feed['Toluene'].flow['Vap','hydrogen'].fix(1e-8)
m.M101.inlet_feed['Toluene'].T.fix(303.2)
m.M101.inlet_feed['Toluene'].P.fix(350000)

m.M101.inlet_feed['Hydrogen'].flow['Liq','benzene'].fix(1e-8)
m.M101.inlet_feed['Hydrogen'].flow['Liq','toluene'].fix(1e-8)
m.M101.inlet_feed['Hydrogen'].flow['Liq','methane'].fix(1e-8)
m.M101.inlet_feed['Hydrogen'].flow['Liq','hydrogen'].fix(1e-8)
m.M101.inlet_feed['Hydrogen'].flow['Vap','benzene'].fix(1e-8)
m.M101.inlet_feed['Hydrogen'].flow['Vap','toluene'].fix(1e-8)
m.M101.inlet_feed['Hydrogen'].flow['Vap','methane'].fix(0.02)
m.M101.inlet_feed['Hydrogen'].flow['Vap','hydrogen'].fix(0.30)
m.M101.inlet_feed['Hydrogen'].T.fix(303.2)
m.M101.inlet_feed['Hydrogen'].P.fix(350000)

In [6]:
degrees_of_freedom(m)

0

In [7]:
seq = SequentialDecomposition()
seq.options.select_tear_method = "heuristic"
seq.options.tear_method = "Wegstein"
seq.options.iterLim = 10

# Using the SD tool
G = seq.create_graph(m)
heuristic_tear_set = seq.tear_set_arcs(G, method="heuristic")
order = seq.calculation_order(G)



In [8]:
for o in heuristic_tear_set:
    print(o.name)

s1


In [9]:
for o in order:
    print(o[0].name)

R101
F101
F102
M101
H101


In [10]:
m.M101.Outlet.pprint()

Outlet : Size=1, Index=None
    Key  : Name : Size : Variable
    None :    P :    1 :    M101.outlet.P
         :    T :    1 :    M101.outlet.T
         : flow :    8 : M101.outlet.flow


In [11]:
def function(unit):
    if 'R' in unit.name:
        Initialize(unit,Arc=m.s1)
    elif 'F101' in unit.name:
        Initialize(unit,Arc=m.s2)
    elif 'F102' in unit.name:
        Initialize(unit,Arc=m.s3)
    elif 'S' in unit.name:
        Initialize(unit,Arc=m.s4)
    elif 'M' in unit.name:
        Initialize(unit,Arc=m.s5)
    elif 'H' in unit.name:
        Initialize(unit,Arc=m.s6)
        
    

In [12]:
value(m.R101.Inlet.T)

298.15

In [13]:
m.R101.Inlet.is_extensive('flow')

False

In [14]:
tear_guesses = {
    'flow': {
        ("Vap", "benzene"): 1e-5,
        ("Vap", "toluene"): 0.3,
        ("Vap", "hydrogen"): 0.30,
        ("Vap", "methane"): 0.02,
        ("Liq", "benzene"): 1e-5,
        ("Liq", "toluene"): 1e-5,
        ("Liq", "hydrogen"): 1e-5,
        ("Liq", "methane"): 1e-5,
    },
    'T': 600,
    'P': 350000,
}

# Pass the tear_guess to the SD tool
seq.set_guesses_for(m.R101.Inlet, tear_guesses)

In [15]:
seq.run(m,function)

model.name="F102";
    - termination condition: infeasible
    - message from solver: Ipopt 3.13.2\x3a Converged to a locally infeasible
      point. Problem may be infeasible.


In [16]:
print('outlet Flow Rate F101:')
for p in m.Phase:
    for c in m.Comp:
        print('\t [',p,',',c,']: ',value(m.F101.outlet.flow[p,c]))

outlet Flow Rate F101:
	 [ Liq , benzene ]:  0.20481274944167296
	 [ Liq , toluene ]:  0.06265706175191449
	 [ Liq , methane ]:  0.0016094351233751111
	 [ Liq , hydrogen ]:  0.00010044843598449738
	 [ Vap , benzene ]:  0.14724296553826283
	 [ Vap , toluene ]:  0.015428031767176597
	 [ Vap , methane ]:  1.263261065263028
	 [ Vap , hydrogen ]:  0.3282113757542806


In [17]:
value(m.M101.outlet.T)

314.0466483072415

In [18]:
print('total Flow Rate M101:')
for p in m.Phase:
    for c in m.Comp:
        print('\t [',p,',',c,']: ',value(m.M101.outlet.flow[p,c]))

total Flow Rate M101:
	 [ Liq , benzene ]:  2.7999999999999986e-08
	 [ Liq , toluene ]:  0.300000018
	 [ Liq , methane ]:  2.7999999999999986e-08
	 [ Liq , hydrogen ]:  2.7999999999999986e-08
	 [ Vap , benzene ]:  0.11779439243061027
	 [ Vap , toluene ]:  0.012342445413741278
	 [ Vap , methane ]:  1.0306088622104224
	 [ Vap , hydrogen ]:  0.5625691106034245


In [19]:
purity = value(m.F102.outlet.flow['Vap','benzene'])/sum(m.F102.outlet.flow['Vap',c] for c in m.Comp)
value(purity)

0.8088958011139135

In [20]:
degrees_of_freedom(m)

0

In [21]:
# 模型求解
solver = SolverFactory('ipopt')
solver.options['tol'] = 1e-4
results = solver.solve(m)
#results = solver.solve(m)

In [9]:
for Unit in m.component_data_objects(Block, active=True):
    print(Unit.name)

R101
H101
F101
s1_expanded
s2_expanded
R101.inlet
R101.outlet
H101.inlet
H101.outlet
F101.inlet
F101.outlet
